In [12]:
import json 
import pandas as pd
from bs4 import BeautifulSoup
import re
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Lambda
from sklearn.model_selection import train_test_split
from transformers import TFBertModel
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
filename_pkl = 'restaurants_reviews.pkl'
filename_tskv = 'geo-reviews-dataset-2023.tskv'
filename_json = 'geo-reviews-dataset-2023.json'

Поскольку по ТЗ даётся json dataset, а мню был найден хороший tskv датасет от Яндекса, то я сначала перевожу tskv в json, а уже потом в pkl ¯\_(ツ)_/¯

In [ ]:
def tsv2json(input_file, output_file):
    arr = []
    
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            items = line.split('\t')
            d = {}
            for item in items:
                key, value = item.split('=', 1)
                d[key.strip()] = value.strip()
            arr.append(d)
    
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(arr, file, ensure_ascii=False, indent=4)

непосредственно функция для перевода tskv в json и запись в файл жсона

In [ ]:
tsv2json(filename_tskv,filename_json)

In [ ]:
df = pd.read_json(filename_json, encoding='utf-8')
df_restaurants = df[df['rubrics'].str.contains('Ресторан', case=False)]

открываем жсон(если мы закрыли Юпитер до этого шага) и переводим его в датафрейм

In [ ]:
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^\w\s]', '', text)  
    text = text.lower()  
    
    return text



функция очистки текста от хтмл тегов, знаков препинания и перевод в нижний регистр

In [ ]:
df_restaurants['text'] = df_restaurants['text'].apply(clean_text)
df_restaurants.drop_duplicates(subset=['text'], inplace=True)
df_restaurants.to_pickle(filename_pkl)


применяем функцию очистки текста к нашему pkl и убираем дубликаты, записываем датафрейм в pkl файл

In [ ]:
def rating_to_label(rating):
    if rating <= 2:
        return 0  
    elif rating >= 4:
        return 1  
    else:
        return None 
df_restaurants['label'] = df_restaurants['rating'].apply(rating_to_label)
df_restaurants = df_restaurants.dropna(subset=['label'])
df_restaurants.to_pickle(filename_pkl)

Функция для присвоения меток нашим отзывам. Если отзыв хороший, то метка 1; если отзыв плохой, то метка 0; если отзыв нейтральный, то дропаем отзыв(кому нужны плохие отзывы? ну и пожалейте мой компьютер, ему еще это обучать)

In [14]:
df_restaurants = pd.read_pickle(filename_pkl)
df_restaurants

,address,name_ru,rating,rubrics,text,label,tokens
0,"Краснодарский край, городской округ Сочи, посё...",Пандок,2,Ресторан,самый большой плюс это месторасположение набер...,0,"[самый, большой, плюс, это, месторасположение,..."
1,"Краснодарский край, городской округ Сочи, посё...",Пандок,5,Ресторан,добрый день сегодня во второй раз посетили с д...,1,"[добрый, день, сегодня, во, второй, раз, посет..."
2,"Московская область, Орехово-Зуевский городской...",Пицца РИО,5,Кафе;Ресторан;Столовая,очень вкусно и по домашнему замечательный комп...,1,"[очень, вкусно, и, по, домашнему, замечательны..."
3,"Омск, улица 70 лет Октября, 19",Корова на бочке,5,"Ресторан;Бар, паб",прекрасное заведение хожу много лет меню не по...,1,"[прекрасное, заведение, хожу, много, лет, меню..."
4,"Ярославль, площадь Труда, 1",IZI kitchen,4,Ресторан;Кафе,все отлично место обстановка обслуживание на в...,1,"[все, отлично, место, обстановка, обслуживание..."
...,...,...,...,...,...,...,...
53701,"Республика Крым, Ялта, набережная имени В.И. Л...",Hispaniola Lounge,5,Ресторан,безумно понравилось это заведение до этого час...,1,"[безумно, понравилось, это, заведение, до, это..."
53702,"Ставрополь, улица Дзержинского, 160",Клюква,5,Ресторан,мне очень понравилосьnnединственный минус жда...,1,"[мне, очень, понравилосьnnединственный, минус,..."
53703,"Москва, улица Юных Ленинцев, 52",Petruccio. Pizza&Pasta,5,Ресторан;Кафе;Пиццерия,очень вкусная пицца нравится их система лояльн...,1,"[очень, вкусная, пицца, нравится, их, система,..."
53704,"Москва, улица Юных Ленинцев, 52",Petruccio. Pizza&Pasta,5,Ресторан;Кафе;Пиццерия,пицца детям понравилась интерьер очень интерес...,1,"[пицца, детям, понравилась, интерьер, очень, и..."


Заново считываем датафрейм(если Ноутбук был закрыт после очистки текста)

In [15]:
texts = df_restaurants['text'].tolist()
labels = df_restaurants['label'].tolist()
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
tokens = tokenizer(texts, padding=True, truncation=True, max_length=100, return_tensors="tf")
input_ids = tokens['input_ids']
attention_masks = tokens['attention_mask']

input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids_np, labels, test_size=0.2, random_state=42)
train_masks, test_masks = train_test_split(
    attention_masks_np, test_size=0.2, random_state=42)

train_inputs = tf.convert_to_tensor(train_inputs)
test_inputs = tf.convert_to_tensor(test_inputs)
train_masks = tf.convert_to_tensor(train_masks)
test_masks = tf.convert_to_tensor(test_masks)
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)


токенизируем текст, делаем падинги до сотки, переводим получившийся текст и метки в нампи(train_test_split отказывается делить тензоры), разбиваем на обучающие и тестовые выборки, возвращаем обратно тензоры

In [ ]:
bert_model = TFBertModel.from_pretrained('DeepPavlov/rubert-base-cased',  from_pt=True)

input_ids = Input(shape=(100,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(100,), dtype=tf.int32, name='attention_mask')

def bert_layer(inputs):
    input_ids, attention_mask = inputs
    return bert_model(input_ids=input_ids, attention_mask=attention_mask)[0]

bert_output = Lambda(bert_layer, output_shape=(100, 768))([input_ids, attention_mask]) 
x = GlobalAveragePooling1D()(bert_output)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

загружаем предобученную модель Берт, получаем токены и маску для них(какие токены следует учитывать), оборачиваем нашу Берту в лямбду(с размерностью (100, 768), где 100 — это количество токенов, а 768 — размер скрытого состояния) и накидываем следующие слои:
1. GlobalAveragePooling1D -  производим выходной тензор размерности (768,). Это позволяет получить одно векторное представление для всей последовательности
2. Dropout(0.3) - случайным образом обнуляем 30% выходных значений. Это помогает предотвратить переобучение модели
3. Dense - выходной  слоей с одним нейроном, функция активации sigmoid. Выдает вероятность между 0 и 1

In [ ]:
model = Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    filepath='model_reviews.keras',  
    save_best_only=True,             
    monitor='val_loss',              
    mode='min',                      
    save_weights_only=False,         
    verbose=1                        
)


history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    validation_data=([test_inputs, test_masks], test_labels),
    epochs=50,
    batch_size=16,
    callbacks=[early_stopping, model_checkpoint],

)

Настраиваем модель с добавлением колбэков early_stopping(обучение остановится, если модель начнет переобучаться) и model_checkpoint(сохранение модели на каждой эпохе), обучаем модель

In [11]:
model = tf.keras.models.load_model('model_reviews.keras', custom_objects={'bert_layer': bert_layer})

загрузка уже обученной нашей модели

In [16]:
bert_model = TFBertModel.from_pretrained('DeepPavlov/rubert-base-cased',  from_pt=True)
loss, accuracy = model.evaluate([test_inputs, test_masks], test_labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

336/336 ━━━━━━━━━━━━━━━━━━━━ 502s 1s/step - accuracy: 0.9583 - loss: 0.1071
Loss: 0.106716088950634
Accuracy: 0.9586669206619263


статистика